## EPC Data Cleaning for Model 2 

This notebook outlines the steps taken to process and clean the EPC records needed for Model 2 in predicting domestic gas and electricity use. 

In [1]:
import pandas as pd
import glob

### Reading EPC data
The model construction uses all EPC records for properties in London. This is public data available to download from here: https://epc.opendatacommunities.org/

In [20]:
#As the EPC records are supplied in read the EPC files in separate files for each local authority, this code reads all csv files in the folder and merges them into a single document
path = "./Data/EnergyData/certificates" 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename,  index_col=None, header=0)
    li.append(df)

EPC = pd.concat(li, axis=0, ignore_index=True)

In [12]:
#Have a look at what information is included in the record
EPC.columns

Index(['LMK_KEY', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE',
       'BUILDING_REFERENCE_NUMBER', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'POTENTIAL_ENERGY_EFFICIENCY', 'PROPERTY_TYPE', 'BUILT_FORM',
       'INSPECTION_DATE', 'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY',
       'LODGEMENT_DATE', 'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT',
       'ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_CURRENT',
       'ENERGY_CONSUMPTION_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISS_CURR_PER_FLOOR_AREA', 'CO2_EMISSIONS_POTENTIAL',
       'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
       'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL',
       'HOT_WATER_COST_CURRENT', 'HOT_WATER_COST_POTENTIAL',
       'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF', 'MAINS_GAS_FLAG', 'FLOOR_LEVEL',
       'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT', 'MAIN_HEATING_CONTROLS',
       'MULTI_GLAZE_PROPORTION', 'GLAZED_TYPE', 'GLAZED_AREA',
      

In [23]:
#drop columns we don't want
EPC=EPC.drop(columns=['POSTTOWN',
       'LODGEMENT_DATETIME', 'TENURE', 'FIXED_LIGHTING_OUTLETS_COUNT',
       'LOW_ENERGY_FIXED_LIGHT_COUNT'])

In [14]:
#define a function which renames column headers of the EPC record
def rename_epcdom(df):
    # rename column headers
    d = {'old': df.columns}
    df_dict = pd.DataFrame(data=d)
    df_dict.set_index('old', inplace=True)

    df_dict['new']=[
            'key',
            'add1',
            'add2',
            'add3',
            'pcode',
            'bref',
            'curr_enr',
            'poten_enr',
            'curr_eff',
            'poten_eff',
            'prop_type',
            'builtform',
            'insp_date',
            'localauth_code',
            'constit_code',
            'county',
            'lodge_date',
            'transact_type',
            'curr_envirr',
            'poten_envirr',
            'curr_encons',
            'poten_encons',
            'curr_co2',
            'curr_co2perarea',
            'poten_co2',
            'curr_light',
            'poten_light',
            'curr_heatcost',
            'poten_heatcost',
            'curr_hotwtr',
            'poten_hotwtr',
            'tfa',
            'tariff',
            'mainsgas',
            'flvl',
            'flattop',
            'flattop_cnt',
            'mainheatcontrol',
            'glaze_percent',
            'glaze_type',
            'glaze_area',
            'nextension',
            'nrooms',
            'nheatedrooms',
            'led_percent',
            'nfireplace',
            'hotwtr',
            'hotwtr_eff',
            'hotwtr_enveff',
            'floor',
            'floor_eff',
            'floor_enveff',
            'window',
            'window_eff',
            'window_enveff',
            'wall',
            'wall_eff',
            'wall_enveff',
            'heat2',
            'heat2_eff',
            'heat2_enveff',
            'roof',
            'roof_eff',
            'roof_enveff',
            'heat',
            'heat_eff',
            'heat_enveff',
            'control',
            'control_eff',
            'control_enveff',
            'light',
            'light_eff',
            'light_enveff',
            'mainfuel',
            'windt',
            'corridor',
            'unhcorridor',
            'fheight',
            'pv',
            'solarwtr',
            'mechvent',
            'addfull',
            'localauth',
            'constit',
            'constructage'
                        ]

    # convert dataframe to dictionary
    col_dict = df_dict['new'].to_dict()
    df.rename(columns = col_dict, inplace=True)
    
    return df, df_dict

def changetypes(df, df_dict):
    # reducing dataframe size by changing column types
    # change to category
    cat = [6, 10, 11, 17, 32, 33, 34, 35, 39, 40, 47, 48, 50,51, 53,54, 56,57, 59,60, 62,63, 65,66, 68,69, 71,72, 75, 79, 80]
    for i in range(len(cat)):
        tempcol = df_dict.new[cat[i]]
        df[tempcol] = df[tempcol].astype('category')

    # change to float32
    floatlist = [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 76, 77]
    for i in range(len(floatlist)):
        tempcol = df_dict.new[floatlist[i]]
        df[tempcol] = df[tempcol].astype('float32')

    # change to int32
    intlist = [8, 18, 19, 20, 21]
    for i in range(len(intlist)):
        tempcol = df_dict.new[intlist[i]]
        df[tempcol] = df[tempcol].astype('int32')

    # change from floating to int32
    f2int = [36, 38, 41, 42, 43, 44, 45, 74, 78] 

    # cannot convert float to int if there are missing values, so first convert NaN to -1
    for i in range(len(f2int)):
        tempcol = df_dict.new[f2int[i]]
        df[tempcol].fillna(-1, inplace=True)
        df[tempcol] = df[tempcol].astype('int32')
    return df


In [51]:
# rename column headers using function defined above
df, df_dict = rename_epcdom(EPC)  
print(df.memory_usage(index=True).sum())

# reduce dataframe size by changing types of columns using function defined above
df = changetypes(df, df_dict)

1141767576


In [46]:
df.shape

(3180384, 85)

In [52]:
# keep only entries of the latest inspection date by converting inspection date column to datetime format and dropping duplicate entries based on building reference and address, and keeping only last ascending date entry 
df['insp_date']= pd.to_datetime(df['insp_date'], format='%Y-%m-%d')
df=df.sort_values(by='insp_date').groupby('bref').tail(1)

In [56]:
df.shape

(2549316, 85)

In [57]:
# map EPC band
epc_dict = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
df['epc_band'] = df.curr_enr.map(epc_dict)
# drop invalid epc entries => drop
df = df[df.epc_band.notnull()]

In [58]:
# map nrooms=-1 to nroom based on floor area bands similar to NEED
import numpy as np
# first drop tfa == 0
df = df[~(df.tfa == 0)]

# create area bins and convert to nrooms for entries of room number with -1
areabins = np.arange(0, 5000, 50).tolist()
arealabels = range(len(areabins)-1)

df['nroomtfa'] = pd.cut(df.tfa, areabins, labels=arealabels)
df['nroom'] = np.where((df.nrooms == -1), df.nroomtfa, df.nrooms)

In [59]:
df['constructage'].unique()
#create new column of simplified age bands and remove the ones with no data

age_mapping = {'England and Wales: before 1900': 0, 'England and Wales: 1900-1929': 1, 'England and Wales: 1930-1949': 2, 'England and Wales: 1950-1966': 3,
                'England and Wales: 1967-1975': 4,'England and Wales: 1976-1982': 5,'England and Wales: 1983-1990': 6,'England and Wales: 1991-1995': 7,'England and Wales: 1996-2002': 8,'England and Wales: 2003-2006': 0,'England and Wales: 2003-2006': 9,'England and Wales: 2007-2011':10,'England and Wales: 2007 onwards':10,'2007':10,'England and Wales: 2012 onwards':11,'2014':11,'2015':11,'2016':11,'2017':11,'2018':11,'2019':11,'2020':11,'2021':11}


df['age'] = df.constructage.map(age_mapping)
#drop rows where age is NaN
df = df[df['age'].notna()]

In [60]:
# convert property type and form to exposed sides 

# create new column for number of exposed sides based on property type and form
prop_mapping = {'House': 0, 'Flat': -2, 'Bungalow': 0.5, 'Maisonette': -2,
                'Park home': 0}
built_mapping = {'Detached': 0, 'Semi-Detached': -1,
                 'End-Terrace': -1, 'Mid-Terrace': -2,
                 'Enclosed Mid-Terrace': -2.5, 'Enclosed End-Terrace': -1.5,
                 'NO DATA!': 0}

df['propmap'] = df.prop_type.map(prop_mapping)
df['builtmap'] = df.builtform.map(built_mapping)
df['exposedsides'] = 6 + df.propmap + df.builtmap

df['type'] = df.prop_type.str.lower()
df['form'] = df.builtform.str.lower()

In [61]:
#map postcodes and LSOAs - this code reads a csv with postocodes and corresponding LSOAs obtained from: https://geoportal.statistics.gov.uk/datasets/06938ffe68de49de98709b0c2ea7c21a/about
dflsoa = pd.read_csv("Data/GeoData/postcodes_lsoa.csv")

C:\Users\signe\anaconda3\envs\dissertation\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
#produce dictionary to match postcodes in EPC record with LSOA
lsoa_dict = pd.Series(dflsoa.lsoa11nm.values, index=dflsoa.pcds).to_dict()

In [63]:
#Map corresponding LSOA to each postcode in EPC record
df['lsoa'] = df.pcode.map(lsoa_dict)
#drop ones that didn't match
df = df[df.lsoa.notnull()]

In [64]:
#now we also want to add IMD to the corresponding LSOA. Data obtained from here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/845345/File_7_-_All_IoD2019_Scores__Ranks__Deciles_and_Population_Denominators_3.csv/preview

dfimd = pd.read_csv("Data/GeoData/imd_lsoa.csv")

In [65]:
#rename columns and create dictionary
dfimd.columns = ['lsoacd', 'lsoa', 'lacd', 'la', 'imdrank', 'imd']
imd_dict = pd.Series(dfimd.imd.values, index=dfimd.lsoa).to_dict()

In [66]:
#map dictionary to epc dataframe
df['imd'] = df.lsoa.map(imd_dict)

In [67]:
#define columns to keep in model
keep_cols = ['bref',
             'epc_band',
             'pcode',
             'lsoa',
             'imd',
             'type',
             'form',
             'exposedsides',
             'tfa',
             'nroom',
             'nrooms',
            'age',
                          ]

df = df[keep_cols]

In [69]:
#write csv to be used further for modelling in R
df.to_csv('Data/EnergyData/cleaned_epc.csv')